# 08 - Explanation System: @func.explain Decorator

## 🧭 Goal

Understand how to make transformations self-documenting using the `@func.explain` decorator pattern.

This notebook will:
- Explain the `@func.explain` decorator pattern
- Show how to attach human-readable explanations to transformations
- Demonstrate template helpers (`purpose_detail_result`, `with_formula`, etc.)
- Pass context via `**kwargs` (plant, asset, pipeline_name, etc.)
- Build transformations with rich, dynamic explanations
- Capture and export explanations to JSON

**Estimated time:** 30 seconds

---

## 🧱 Core Concepts

**The @func.explain Pattern:**
```python
@transformation("my_transform")
def my_transform(df, threshold):
    return df[df.value > threshold]

@my_transform.explain
def explain(threshold, **context):
    plant = context.get('plant', 'Unknown')
    return f"Filter {plant} records above {threshold}"

# Get explanation with context
text = my_transform.get_explanation(
    threshold=100, 
    plant="NKC", 
    asset="Germ Dryer 1"
)
```

**Template Helpers:**
- `purpose_detail_result()` - Standard Purpose/Details/Result format
- `with_formula()` - Include mathematical formulas
- `simple_explanation()` - One-liner explanations

**Context Passing:**
- Explanations receive `**kwargs` with pipeline metadata
- Common context: `plant`, `asset`, `pipeline_name`, `layer`, `environment`
- Makes explanations dynamic and context-aware

## 🔧 Setup

In [ ]:
# ✅ Environment Setup
import sys
import os
from pathlib import Path
import pandas as pd
import json

# Navigate to project root
project_root = Path.cwd().parent if Path.cwd().name == 'walkthroughs' else Path.cwd()
os.chdir(project_root)

# Create artifacts directory
artifacts_dir = Path('walkthroughs/.artifacts/08_explanation')
artifacts_dir.mkdir(parents=True, exist_ok=True)

# Import ODIBI explanation system
from odibi.transformations import transformation, get_registry
from odibi.transformations.templates import (
    purpose_detail_result,
    with_formula,
    simple_explanation,
    table_explanation
)

# Clear registry for clean test
get_registry().clear()

print(f"✅ Environment ready")
print(f"📁 Artifacts: {artifacts_dir}")

## ▶️ Run: Create Transformation with @func.explain

In [ ]:
# Define transformation with explanation
@transformation("filter_threshold", version="1.0.0", category="filtering")
def filter_threshold(df, column, threshold, operation="greater_than"):
    """Filter DataFrame rows based on threshold."""
    if operation == "greater_than":
        return df[df[column] > threshold]
    elif operation == "less_than":
        return df[df[column] < threshold]
    elif operation == "equals":
        return df[df[column] == threshold]
    else:
        raise ValueError(f"Unknown operation: {operation}")

@filter_threshold.explain
def explain(column, threshold, operation="greater_than", **context):
    """Generate explanation for filter_threshold transformation."""
    plant = context.get('plant', 'Unknown Plant')
    asset = context.get('asset', 'Unknown Asset')
    
    op_text = {
        "greater_than": "greater than",
        "less_than": "less than",
        "equals": "equal to"
    }.get(operation, operation)
    
    return purpose_detail_result(
        purpose=f"Filter records for {plant} - {asset}",
        details=[
            f"Column: {column}",
            f"Condition: values {op_text} {threshold}",
            f"Operation: {operation}"
        ],
        result=f"Filtered dataset with only records where {column} is {op_text} {threshold}"
    )

print("✅ Created transformation: 'filter_threshold'")
print(f"   Has explanation: {filter_threshold.has_explanation()}")

# Get explanation with context
explanation = filter_threshold.get_explanation(
    column="temperature",
    threshold=100,
    operation="greater_than",
    plant="Test Plant",
    asset="Asset1"
)

print("\n📋 Explanation:\n")
print(explanation)

## 🎨 Create: Transformation with Formula Template

In [ ]:
# Create efficiency calculation with formula
@transformation("calculate_efficiency", version="1.0.0", category="calculations")
def calculate_efficiency(df, input_col, output_col, efficiency_col="efficiency"):
    """Calculate efficiency as output/input ratio."""
    df = df.copy()
    df[efficiency_col] = (df[output_col] / df[input_col]) * 100
    return df

@calculate_efficiency.explain
def explain(input_col, output_col, efficiency_col="efficiency", **context):
    """Explain efficiency calculation."""
    plant = context.get('plant', 'Unknown Plant')
    asset = context.get('asset', 'Unknown Asset')
    pipeline = context.get('pipeline_name', 'Unknown Pipeline')
    
    return with_formula(
        purpose=f"Calculate efficiency for {plant} - {asset} in {pipeline}",
        details=[
            f"Input measure: {input_col}",
            f"Output measure: {output_col}",
            f"Result column: {efficiency_col}",
            "Units: Percentage (%)"
        ],
        formula=f"{efficiency_col} = ({output_col} / {input_col}) × 100",
        result=f"New column '{efficiency_col}' added with efficiency percentage"
    )

print("✅ Created transformation: 'calculate_efficiency'")

# Get explanation with context
efficiency_explanation = calculate_efficiency.get_explanation(
    input_col="natural_gas",
    output_col="steam",
    efficiency_col="thermal_efficiency",
    plant="NKC",
    asset="Germ Dryer 1",
    pipeline_name="efficiency_monitoring"
)

print("\n📋 Efficiency Explanation:\n")
print(efficiency_explanation)

## 🔍 Inspect: Simple Explanation Template

In [ ]:
# Create simple transformation with one-liner explanation
@transformation("drop_nulls", version="1.0.0", category="cleaning")
def drop_nulls(df, columns=None):
    """Drop rows with null values."""
    if columns:
        return df.dropna(subset=columns)
    return df.dropna()

@drop_nulls.explain
def explain(columns=None, **context):
    """Explain null removal."""
    if columns:
        col_list = ", ".join(columns) if isinstance(columns, list) else columns
        return simple_explanation(
            purpose=f"Remove rows with missing values in: {col_list}",
            result="Clean dataset without nulls in specified columns"
        )
    else:
        return simple_explanation(
            purpose="Remove all rows containing any missing values",
            result="Clean dataset with complete records only"
        )

print("✅ Created transformation: 'drop_nulls'")

# Get explanation without columns
explanation1 = drop_nulls.get_explanation()
print("\n📋 Explanation (all columns):\n")
print(explanation1)

# Get explanation with specific columns
explanation2 = drop_nulls.get_explanation(columns=["temperature", "pressure"])
print("\n📋 Explanation (specific columns):\n")
print(explanation2)

## 🧪 Test: Export Explanations to JSON

In [ ]:
# Collect explanations with different contexts
explanations_data = []

# Filter threshold explanation
explanations_data.append({
    "transformation": "filter_threshold",
    "explanation": filter_threshold.get_explanation(
        column="temperature",
        threshold=100,
        operation="greater_than",
        plant="Test Plant",
        asset="Asset1"
    ),
    "context": {
        "plant": "Test Plant",
        "asset": "Asset1",
        "column": "temperature",
        "threshold": 100
    },
    "template_used": "purpose_detail_result"
})

# Efficiency calculation explanation
explanations_data.append({
    "transformation": "calculate_efficiency",
    "explanation": calculate_efficiency.get_explanation(
        input_col="fuel",
        output_col="energy",
        plant="NKC",
        asset="Boiler 2",
        pipeline_name="energy_analytics"
    ),
    "context": {
        "plant": "NKC",
        "asset": "Boiler 2",
        "pipeline_name": "energy_analytics",
        "input_col": "fuel",
        "output_col": "energy"
    },
    "template_used": "with_formula"
})

# Drop nulls explanation
explanations_data.append({
    "transformation": "drop_nulls",
    "explanation": drop_nulls.get_explanation(
        columns=["temperature", "pressure"]
    ),
    "context": {
        "columns": ["temperature", "pressure"]
    },
    "template_used": "simple_explanation"
})

# Save to JSON
with open(artifacts_dir / 'explanations.json', 'w') as f:
    json.dump(explanations_data, f, indent=2)

print(f"✅ Exported {len(explanations_data)} explanations to:")
print(f"   {artifacts_dir / 'explanations.json'}")

# Display summary
print("\n📊 Explanations Summary:")
for item in explanations_data:
    print(f"\n  • {item['transformation']}")
    print(f"    Template: {item['template_used']}")
    print(f"    Context keys: {', '.join(item['context'].keys())}")

## 🎯 Practical: Build Transformation with Rich Context

In [ ]:
# Create a transformation with rich, context-aware explanation
@transformation("aggregate_metrics", version="1.0.0", category="aggregation")
def aggregate_metrics(df, group_cols, agg_cols, agg_func="mean"):
    """Aggregate metrics by groups."""
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    
    agg_dict = {col: agg_func for col in agg_cols}
    return df.groupby(group_cols).agg(agg_dict).reset_index()

@aggregate_metrics.explain
def explain(group_cols, agg_cols, agg_func="mean", **context):
    """Generate rich explanation with full context."""
    # Extract context
    plant = context.get('plant', 'Unknown Plant')
    asset = context.get('asset', 'Unknown Asset')
    pipeline = context.get('pipeline_name', 'Unknown Pipeline')
    layer = context.get('layer', 'Unknown Layer')
    environment = context.get('environment', 'development')
    
    # Format lists
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    if isinstance(agg_cols, str):
        agg_cols = [agg_cols]
    
    group_list = ", ".join(group_cols)
    agg_list = ", ".join(agg_cols)
    
    # Build detailed explanation
    details = [
        f"Environment: {environment}",
        f"Plant: {plant}",
        f"Asset: {asset}",
        f"Pipeline: {pipeline}",
        f"Layer: {layer}",
        f"Grouping by: {group_list}",
        f"Aggregating: {agg_list}",
        f"Function: {agg_func}"
    ]
    
    return purpose_detail_result(
        purpose=f"Aggregate {agg_list} metrics for {plant} - {asset}",
        details=details,
        result=f"Grouped dataset with {agg_func} values per {group_list}"
    )

print("✅ Created transformation: 'aggregate_metrics'")

# Generate explanation with full context
rich_explanation = aggregate_metrics.get_explanation(
    group_cols=["date", "shift"],
    agg_cols=["temperature", "pressure", "flow_rate"],
    agg_func="mean",
    plant="NKC",
    asset="Germ Dryer 1",
    pipeline_name="daily_metrics",
    layer="silver",
    environment="production"
)

print("\n📋 Rich Explanation with Full Context:\n")
print(rich_explanation)

# Save to file
transformation_data = {
    "transformation": "aggregate_metrics",
    "explanation": rich_explanation,
    "context": {
        "group_cols": ["date", "shift"],
        "agg_cols": ["temperature", "pressure", "flow_rate"],
        "agg_func": "mean",
        "plant": "NKC",
        "asset": "Germ Dryer 1",
        "pipeline_name": "daily_metrics",
        "layer": "silver",
        "environment": "production"
    },
    "template_used": "purpose_detail_result",
    "has_explanation": aggregate_metrics.has_explanation()
}

with open(artifacts_dir / 'transformation_with_context.json', 'w') as f:
    json.dump(transformation_data, f, indent=2)

print(f"\n✅ Saved rich context example to:")
print(f"   {artifacts_dir / 'transformation_with_context.json'}")

## 📄 Export: Create Markdown Documentation

In [ ]:
# Generate markdown documentation of all explanations
markdown_content = "# Transformation Explanations\n\n"
markdown_content += "Auto-generated documentation of transformations with explanations.\n\n"
markdown_content += "---\n\n"

# List all transformations with explanations
registry = get_registry()
transformations_with_explanations = []

for name in registry.list():
    func = registry.get(name)
    if hasattr(func, 'has_explanation') and func.has_explanation():
        transformations_with_explanations.append(name)

markdown_content += f"## Summary\n\n"
markdown_content += f"Found {len(transformations_with_explanations)} transformations with explanations:\n\n"

for name in transformations_with_explanations:
    markdown_content += f"- `{name}`\n"

markdown_content += "\n---\n\n"

# Add detailed examples
markdown_content += "## Example Explanations\n\n"

# Example 1: Filter with context
markdown_content += "### 1. Filter Threshold\n\n"
markdown_content += "```python\n"
markdown_content += "filter_threshold(df, column='temperature', threshold=100, operation='greater_than')\n"
markdown_content += "```\n\n"
markdown_content += filter_threshold.get_explanation(
    column="temperature",
    threshold=100,
    operation="greater_than",
    plant="Test Plant",
    asset="Asset1"
)
markdown_content += "\n\n---\n\n"

# Example 2: Efficiency calculation
markdown_content += "### 2. Calculate Efficiency\n\n"
markdown_content += "```python\n"
markdown_content += "calculate_efficiency(df, input_col='fuel', output_col='energy')\n"
markdown_content += "```\n\n"
markdown_content += efficiency_explanation
markdown_content += "\n\n---\n\n"

# Example 3: Aggregate metrics
markdown_content += "### 3. Aggregate Metrics\n\n"
markdown_content += "```python\n"
markdown_content += "aggregate_metrics(df, group_cols=['date', 'shift'], agg_cols=['temperature'], agg_func='mean')\n"
markdown_content += "```\n\n"
markdown_content += rich_explanation
markdown_content += "\n\n---\n\n"

# Add template reference
markdown_content += "## Template Helpers\n\n"
markdown_content += "ODIBI provides helper functions for common explanation patterns:\n\n"
markdown_content += "- **`purpose_detail_result()`** - Standard Purpose/Details/Result format\n"
markdown_content += "- **`with_formula()`** - Include mathematical formulas\n"
markdown_content += "- **`simple_explanation()`** - One-liner explanations\n"
markdown_content += "- **`table_explanation()`** - Explanations with embedded tables\n\n"

markdown_content += "### Example Usage\n\n"
markdown_content += "```python\n"
markdown_content += "from odibi.transformations.templates import purpose_detail_result\n\n"
markdown_content += "@my_transform.explain\n"
markdown_content += "def explain(param1, **context):\n"
markdown_content += "    return purpose_detail_result(\n"
markdown_content += "        purpose=\"Transform data\",\n"
markdown_content += "        details=[\"Step 1\", \"Step 2\"],\n"
markdown_content += "        result=\"Transformed dataset\"\n"
markdown_content += "    )\n"
markdown_content += "```\n"

# Save markdown
with open(artifacts_dir / 'explanation_examples.md', 'w') as f:
    f.write(markdown_content)

print(f"✅ Generated markdown documentation:")
print(f"   {artifacts_dir / 'explanation_examples.md'}")
print(f"\n📊 Documented {len(transformations_with_explanations)} transformations")

## ✅ Self-Check

In [ ]:
import time
start_time = time.time()

try:
    # Check artifact files exist
    assert (artifacts_dir / 'explanations.json').exists(), "explanations.json not found"
    assert (artifacts_dir / 'transformation_with_context.json').exists(), "transformation_with_context.json not found"
    assert (artifacts_dir / 'explanation_examples.md').exists(), "explanation_examples.md not found"
    
    # Load and validate explanations.json
    with open(artifacts_dir / 'explanations.json') as f:
        explanations = json.load(f)
    
    assert len(explanations) >= 3, f"Expected at least 3 explanations, got {len(explanations)}"
    
    # Check required fields in each explanation
    for exp in explanations:
        assert 'transformation' in exp, "Missing 'transformation' field"
        assert 'explanation' in exp, "Missing 'explanation' field"
        assert 'context' in exp, "Missing 'context' field"
        assert 'template_used' in exp, "Missing 'template_used' field"
        assert len(exp['explanation']) > 0, "Empty explanation text"
    
    # Verify at least one explanation uses each template type
    templates_used = {exp['template_used'] for exp in explanations}
    assert 'purpose_detail_result' in templates_used, "No explanation uses purpose_detail_result"
    assert len(templates_used) >= 2, f"Expected at least 2 different templates, got {len(templates_used)}"
    
    # Validate transformation_with_context.json
    with open(artifacts_dir / 'transformation_with_context.json') as f:
        context_data = json.load(f)
    
    assert 'transformation' in context_data, "Missing transformation field"
    assert 'context' in context_data, "Missing context field"
    assert 'plant' in context_data['context'], "Missing plant in context"
    assert 'asset' in context_data['context'], "Missing asset in context"
    
    # Check markdown file
    with open(artifacts_dir / 'explanation_examples.md') as f:
        markdown = f.read()
    
    assert len(markdown) > 100, "Markdown file is too short"
    assert '# Transformation Explanations' in markdown, "Missing header"
    assert 'purpose_detail_result' in markdown, "Missing template reference"
    
    # Verify transformations are callable
    df_test = pd.DataFrame({'value': [1, 2, 3, 4, 5]})
    result = filter_threshold(df_test, column='value', threshold=2, operation='greater_than')
    assert len(result) == 3, "filter_threshold not working correctly"
    
    # Verify explanations are accessible
    assert filter_threshold.has_explanation(), "filter_threshold missing explanation"
    assert calculate_efficiency.has_explanation(), "calculate_efficiency missing explanation"
    assert drop_nulls.has_explanation(), "drop_nulls missing explanation"
    assert aggregate_metrics.has_explanation(), "aggregate_metrics missing explanation"
    
    # Check runtime
    elapsed = time.time() - start_time
    assert elapsed < 30, f"Runtime {elapsed:.1f}s exceeds 30s budget"
    
    print("🎉 Walkthrough verified successfully!")
    print(f"⏱️  Runtime: {elapsed:.2f}s")
    print(f"📊 Explanations: {len(explanations)}")
    print(f"📋 Templates used: {', '.join(templates_used)}")
    print(f"✅ All checks passed!")
    
except AssertionError as e:
    print(f"❌ Walkthrough failed: {e}")
    raise
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

## 🧠 Reflection

### What You Learned

1. **@func.explain Pattern**: Transformations can self-document using the `@func.explain` decorator
2. **Template Helpers**: ODIBI provides reusable templates for common explanation patterns
3. **Context Passing**: Explanations receive pipeline metadata via `**kwargs` for dynamic, context-aware text
4. **Rich Documentation**: Explanations can include formulas, tables, and detailed context

### Where This Fits in ODIBI

```
Transformation Lifecycle:
@transformation → Register in registry → Execute on data
     ↓
@func.explain  → Register explanation → Generate docs/logs
     ↓
get_explanation(context) → Human-readable text
```

The explanation system makes transformations **self-documenting**. Every operation can explain what it does, using pipeline context to generate relevant, human-readable descriptions.

### Key Insights

- **Separation of Concerns**: Logic and documentation are separate but linked
- **Dynamic Content**: Explanations adapt based on context (plant, asset, etc.)
- **Template Reuse**: Common patterns avoid duplication
- **Machine-Readable**: Explanations can be exported to JSON for automated documentation

### Practical Applications

- **Pipeline Documentation**: Auto-generate docs from running pipelines
- **Audit Trails**: Explain what transformations did with actual parameters
- **User Interfaces**: Show explanations to non-technical users
- **Debugging**: Understand what a transformation does without reading code

---

## ⏭ Next Steps

**Explore:**
- Read `odibi/transformations/explanation.py` - The explanation decorator implementation
- Read `odibi/transformations/templates.py` - All available template helpers
- Read `odibi/transformations/context.py` - Context passing system
- Check `tests/unit/test_explanation.py` - More usage examples

**Try:**
- Add explanations to your own transformations
- Create custom template helpers for your use case
- Build an auto-documentation system that exports all explanations
- Integrate explanations into pipeline execution logs